In [1]:
# reveal.js presentation configuration
from notebook.services.config import ConfigManager

cm = ConfigManager()
cm.update('livereveal', {
              'theme': 'league',
              'transition': 'fade',
              'center': 'false',
              'overview' : 'true',
              'start_slideshow_at': 'selected'
})

# imports
import theano
from theano import tensor
import codecs
import numpy
import sys
from blocks import initialization
from blocks import roles
from blocks.model import Model
from blocks.bricks import Linear, NDimensionalSoftmax
from blocks.bricks.parallel import Fork
from blocks.bricks.recurrent import GatedRecurrent
from blocks.bricks.lookup import LookupTable
from blocks.filter import VariableFilter
from blocks.serialization import load_parameters
from blocks.bricks import NDimensionalSoftmax

# Language modeling with RNN

[Fabio A. González](http://dis.unal.edu.co/~fgonza/), Universidad Nacional de Colombia

## Setup
* Training data: Biblia Reina Valera 1960
* Software:
  * [Blocks](https://github.com/mila-udem/blocks): "Blocks is a framework that helps you build neural network models on top of Theano"
  * [Theano](http://deeplearning.net/software/theano/): "Theano is a Python library that allows you to define, optimize, and evaluate mathematical expressions involving multi-dimensional arrays efficiently"

## Training data

In [2]:
# Load training file to get vocabulary
text_file = 'biblia.txt' # input file
with codecs.open(text_file, 'r', 'utf-8') as f:
    data = f.read()
    
chars = list(set(data))
vocab_size = len(chars)
char_to_ix = {ch: i for i, ch in enumerate(chars)}
ix_to_char = {i: ch for i, ch in enumerate(chars)}
print "Total number of chars:", len(data)
print "Vocabulary size:", vocab_size

Total number of chars: 978848
Vocabulary size: 85


## Training data

In [3]:
print data[21000:22000]

s será medido. 3 ¿Y por qué miras la paja que está en el ojo de tu hermano, y no echas de ver la viga que está en tu propio ojo? 4 ¿O cómo dirás a tu hermano: Déjame sacar la paja de tu ojo, y he aquí la viga en el ojo tuyo? 5 ¡Hipócrita! saca primero la viga de tu propio ojo, y entonces verás bien para sacar la paja del ojo de tu hermano.

6 No deis lo santo a los perros, ni echéis vuestras perlas delante de los cerdos, no sea que las pisoteen, y se vuelvan y os despedacen.

La oración, y la regla de oro

(Lc. 11.9-13; 6.31)

7 Pedid, y se os dará; buscad, y hallaréis; llamad, y se os abrirá. 8 Porque todo aquel que pide, recibe; y el que busca, halla; y al que llama, se le abrirá. 9 ¿Qué hombre hay de vosotros, que si su hijo le pide pan, le dará una piedra? 10 ¿O si le pide un pescado, le dará una serpiente? 11 Pues si vosotros, siendo malos, sabéis dar buenas dádivas a vuestros hijos, ¿cuánto más vuestro Padre que está en los cielos dará buenas cosas a los que le pidan? 12 


## Network architecture

<img src="rnn_architecture.jpg" width= 400>

## Define the layers

In [4]:
# Define the model structure
embedding_size = 256 # number of hidden units per layer

# Input
lookup = LookupTable(length=vocab_size, dim=embedding_size)

# Layer 1
fork1 = Fork(output_names=['linear1', 'gates1'],
             input_dim=embedding_size, output_dims=[embedding_size, embedding_size * 2])
fork1.name = 'fork1'
grnn1 = GatedRecurrent(dim=embedding_size)
grnn1.name = 'grnn1'

# Layer 2
fork2 = Fork(output_names=['linear2', 'gates2'],
             input_dim=embedding_size, output_dims=[embedding_size, embedding_size * 2])
fork2.name = 'fork2'
grnn2 = GatedRecurrent(dim=embedding_size)
grnn2.name = 'grnn2'

# Softmax layer
hidden_to_output = Linear(name='hidden_to_output', input_dim=embedding_size,
                          output_dim=vocab_size)
softmax = NDimensionalSoftmax()

## Connect the layers

In [5]:
# Propagate x until top brick to get y_hat predictions
x = tensor.imatrix('features')  # input
y = tensor.imatrix('targets')   # output
embedding = lookup.apply(x)
linear1, gates1 = fork1.apply(embedding)
h1 = grnn1.apply(linear1, gates1)
h1.name = 'h1'
linear2, gates2 = fork2.apply(h1)
h2 = grnn2.apply(linear2, gates2)
h2.name = 'h2'
linear3 = hidden_to_output.apply(h2)
linear3.name = 'linear3'
y_hat = softmax.apply(linear3, extra_ndim=1)
y_hat.name = 'y_hat'

# COST
cost = softmax.categorical_cross_entropy(y, linear3, extra_ndim=1).mean()
cost.name = 'cost'

model = Model(cost)

## Load parameters and build Theano graph

In [6]:
# Load model parameters from a file
with open('grnn_best.tar') as model_file:
    model_params = model.get_parameter_dict().keys()
    param_vals = {k:v for k,v in load_parameters(model_file).iteritems() if k in model_params}
    model.set_parameter_values(param_vals)

In [7]:
# Define Theano graph
y, x = model.inputs
softmax = NDimensionalSoftmax()
linear_output = [v for v in model.variables if v.name == 'linear3'][0]
y_hat = softmax.apply(linear_output, extra_ndim=1)
predict = theano.function([x], y_hat)
#theano.printing.pydotprint(predict, outfile="theano_graph.svg", format = 'svg', var_with_name_simple=True)  

<img src="theano_graph.svg">

In [8]:
#take activations of last element
activations = [h1[-1].flatten(), h2[-1].flatten()]
initial_states = [grnn1.parameters[-1], grnn2.parameters[-1]]
states_as_params = [tensor.vector(dtype=initial.dtype) for initial in initial_states]

#Get prob. distribution of the last element in the last seq of the batch
fprop = theano.function([x] + states_as_params, activations + [y_hat[-1, -1, :]], givens=zip(initial_states, states_as_params))

def sample(x_curr, states_values, fprop, temperature=1.0):
    '''
    Propagate x_curr sequence and sample next element according to
    temperature sampling.
    Return: sampled element and a list of the hidden activations produced by fprop.
    '''
    activations = fprop(x_curr, *states_values)
    probs = activations.pop().astype('float64')
    probs = probs / probs.sum()
    if numpy.random.binomial(1, temperature) == 1:
        sample = numpy.random.multinomial(1, probs).nonzero()[0][0]
    else:
        sample = probs.argmax()

    return sample, activations, probs[sample]

def init_params(ix_to_char, primetext=u''):
    if not primetext or len(primetext) == 0:
        primetext = ix_to_char[numpy.random.randint(vocab_size)]
    primetext = ''.join([ch for ch in primetext if ch in char_to_ix.keys()])
    if len(primetext) == 0:
        raise Exception('primetext characters are not in the vocabulary')
    x_curr = numpy.expand_dims(
        numpy.array([char_to_ix[ch] for ch in primetext], dtype='uint8'), axis=1)

    states_values = [initial.get_value() for initial in initial_states]
    return x_curr, states_values
    
def stochastic_sampling(length, ix_to_char, primetext=u'', temperature=1.0):
    x_curr, states_values = init_params(ix_to_char, primetext)
    sys.stdout.write('Starting sampling\n' + primetext)
    for _ in range(length):
        idx, states_values, probs = sample(x_curr, states_values, fprop, temperature)
        sys.stdout.write(ix_to_char[idx])
        x_curr = [[idx]]

    sys.stdout.write('\n')

def beam_sampling(length, primetext=u'', beam_size=5, temperature=1.0):
    x_curr, states_values = init_params(ix_to_char, primetext)
    inputs = [x_curr] * beam_size
    states = [states_values] * beam_size
    logprobs = numpy.zeros((beam_size, 1))
    seqs = numpy.zeros((length+x_curr.shape[0], beam_size))
    seqs[0:x_curr.shape[0], :] = numpy.repeat(x_curr, beam_size, axis=1)
    for k in range(length):
        probs = numpy.zeros((beam_size,beam_size))
        indices = numpy.zeros((beam_size,beam_size), dtype='int32')
        hstates = numpy.empty((beam_size,beam_size), dtype=list)
        for i in range(beam_size):
            for j in range(beam_size):
                indices[i][j], hstates[i][j], probs[i][j] = sample(inputs[i], states[i], fprop, temperature)
        probs = numpy.log(probs) + logprobs
        best_idx = probs.argmax(axis=1)
        inputs = [[[idx]] for idx in indices[range(beam_size), best_idx]]
        states = [hs for hs in hstates[range(beam_size), best_idx]]
        logprobs = probs[range(beam_size), best_idx].reshape((beam_size, 1))
        seqs[k +x_curr.shape[0], :] = numpy.array(inputs).flatten()

    return logprobs.flatten(), numpy.array(seqs).squeeze()

In [9]:
logprobs, seqs = beam_sampling(50, primetext=u'')
for i in logprobs.flatten().argsort()[::-1]:
    print 'log P(s) = {0:3.3f}. Sample: '.format(logprobs.flatten()[i]) + u''.join([ix_to_char[ix] for ix in numpy.array(seqs).squeeze()[:,i]])
    print '~' * 50

log P(s) = -26.658. Sample: , y le habían sido después de estas cosas, para que
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
log P(s) = -27.567. Sample: , y le preguntaron, de los demonios en la palabra d
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
log P(s) = -27.903. Sample: , y le dijo: Señor, ¿por qué nos ha de los demonios
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
log P(s) = -31.325. Sample: , y le dijo: ¿Qué presentaron a la verdad, y le dij
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
log P(s) = -32.353. Sample: , y le dijo: ¿Qué harán la carne, y no se levantará
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


## Sampling from the model

* The model calculates the probability of the next word given the previous words:  
$$P(w_t | w_{t-1}, w_{t-2},\dots, w_{1})$$
* We sample from the model using this conditional probability
  ```python
  for i in [1..n]:
      P = predict_next() 
      bin_var = sample_binomial(temperature)
      if bin_var:
          w_i = sample_multinomial(P) 
      else:
          w_i = P.argmax() 
  ```

In [10]:
stochastic_sampling(3000, ix_to_char, primetext=u'Y Jesús dijo', temperature=0.3)

Starting sampling
Y Jesús dijo: ¿Quién es el que había de la contada de la carne, y os digo que no se había una para la palabra de Esteban las cosas o por las que estaban contigo, de la cual deseación predicaba de la común al que hablaba de la resurrección de la casa de las cosas que están en Si. 17 Entonces los que habían salido de la carne, y a la hora a nadie dice: 


Viene a la multitud, dando los del cielo, y levantarán en la carne, y le dijo: ¿Por qué no te había de la carne, pero no se lo que había sido dicho estas cosas, y le predicen con él, diciendo: ¿Quién fue llevado en tierra, de la ciudad se levantó de la ciudad de las iglesias de la tribulacione de vuestros padres, y vean. 

42 Entonces el que había dado a su propia otra vez a los que presentanos de los muertos, por cuanto es el que había siete esemos hay de los que con él estaban con las discípulos, y le dijeron: Maestro, Pablo. 17 Porque el que llama de la cárcel, y los demonios de los que le dijeron: ¿Quién es el que 

## Probability of a text
* The probability of a text is:  
$$P(w_1, \dots, w_n) = P(w_1)\prod_{i=2}^{n}\ P(w_i | w_{i-1},\dots, w_{1})$$


In [11]:
# Function to calculate the probability of a text
def log_likelihood(text):
    text = ''.join([ch for ch in text if ch in char_to_ix])
    x_curr = numpy.expand_dims(numpy.array([char_to_ix[ch] for ch in text], dtype='uint8'), axis=1)
    probs = predict(x_curr).squeeze()
    return sum([numpy.log(probs[i,c]) for i,c in enumerate(x_curr[1:].flatten())])

## Most likely phrases from a bag of words

In [12]:
from itertools import permutations
bow =  [u' ', 'hombre', 'ama', 'la', 'a']
perms = [' '.join(perm) for perm in permutations(bow)]
for p, t in sorted([(-log_likelihood(text),text) for text in perms])[:20]:
    print p, t

31.8389759056 hombre ama a la  
32.0275349372   la hombre ama a
32.3668355764   hombre ama a la
32.3860355795 la hombre ama a  
32.7130291137   ama a la hombre
33.0558109689   a la hombre ama
33.7818600005   ama la hombre a
33.8319883609   la ama a hombre
34.0099810799   hombre a la ama
34.6731117631   a hombre ama la
34.7088414131   ama a hombre la
34.7305872297   a hombre la ama
35.1243224444 ama a la hombre  
35.4159899969 a la hombre ama  
35.4609327316   la hombre a ama
35.7856470298 la ama a hombre  
36.0883222858   ama hombre a la
36.1893816709   hombre ama la a
36.7862393504   hombre la ama a
37.1080666702   la ama hombre a


## Least likely phrases 

In [13]:
perms = [' '.join(perm) for perm in permutations(bow)]
for p, t in sorted([(-log_likelihood(text),text) for text in perms])[-20:]:
    print p, t

51.8239402816 a hombre   la ama
51.958106713 la a hombre   ama
52.066835979 ama la   hombre a
52.2481978757 la   hombre a ama
52.4202299064 ama la   a hombre
52.5298534369 la a   hombre ama
52.830648843 a ama la   hombre
52.8850183918 ama hombre la   a
53.6537188433 hombre a ama   la
54.0823958933 ama   la a hombre
54.4299677624 hombre   la a ama
54.7567527487 hombre la   a ama
54.9093022025 ama hombre   la a
54.9575976433 ama hombre a   la
56.0846975897 la a ama   hombre
56.1855978359 a ama hombre   la
56.236034771 la   a ama hombre
56.5873797276 hombre la a   ama
56.740208019 la a   ama hombre
59.1151459806 ama la a   hombre


## Morphology

In [14]:
from itertools import permutations
from random import shuffle


In [15]:
text = list(u'mnp')
perms = [''.join(perm) for perm in permutations(text)]
for p, t in sorted([(-log_likelihood(text),text) for text in perms])[:5]:
    print p, t
print "------------------"
for p, t in sorted([(-log_likelihood(text),text) for text in perms])[-5:]:
    print p, t

10.2908600823 mpn
11.7298278541 mnp
13.2396000968 nmp
18.4572623821 npm
22.6503256425 pmn
------------------
11.7298278541 mnp
13.2396000968 nmp
18.4572623821 npm
22.6503256425 pmn
23.4249039468 pnm


## Structure

In [16]:
print stochastic_sampling(400, ix_to_char, u"(Lc. ", temperature = 0.3)

Starting sampling
(Lc. 11.1-9)

13 Su madre de vosotros era como si no prometinado a Cristo por la verdad, y le dijo: ¿Qué había de destrucción, sino que me ha de en la ciudad, y en fue de la tierra, y les dijo: ¿Qué había de la hora estaba en las cuales nuestro Señor vuestra promesa de la carne, y le dijo: ¿Qué había de la tierra, 
Y le dijo: Por tanto, de manera que se letras venido de la fe de la carne, y le diste
None


## A model trained from a different dataset

In [17]:
# Load training file to get vocabulary
text_file = 'reg1.txt' # input file
with codecs.open(text_file, 'r', 'utf-8') as f:
    data = f.read()
chars = list(set(data))
vocab_size = len(chars)
char_to_ix = {ch: i for i, ch in enumerate(chars)}
ix_to_char = {i: ch for i, ch in enumerate(chars)}
print "Total number of chars:", len(data)
print "Vocabulary size:", vocab_size

Total number of chars: 11120585
Vocabulary size: 152


In [18]:
# Input
lookup = LookupTable(length=vocab_size, dim=embedding_size)
# Layer 1
fork1 = Fork(output_names=['linear1', 'gates1'],
             input_dim=embedding_size, output_dims=[embedding_size, embedding_size * 2])
fork1.name = 'fork1'
grnn1 = GatedRecurrent(dim=embedding_size)
grnn1.name = 'grnn1'
# Layer 2
fork2 = Fork(output_names=['linear2', 'gates2'],
             input_dim=embedding_size, output_dims=[embedding_size, embedding_size * 2])
fork2.name = 'fork2'
grnn2 = GatedRecurrent(dim=embedding_size)
grnn2.name = 'grnn2'
# Softmax layer
hidden_to_output = Linear(name='hidden_to_output', input_dim=embedding_size,
                          output_dim=vocab_size)
softmax = NDimensionalSoftmax()

# Propagate x until top brick to get y_hat predictions
x = tensor.imatrix('features')  # input
y = tensor.imatrix('targets')   # output
embedding = lookup.apply(x)
linear1, gates1 = fork1.apply(embedding)
h1 = grnn1.apply(linear1, gates1)
h1.name = 'h1'
linear2, gates2 = fork2.apply(h1)
h2 = grnn2.apply(linear2, gates2)
h2.name = 'h2'
linear3 = hidden_to_output.apply(h2)
linear3.name = 'linear3'
y_hat = softmax.apply(linear3, extra_ndim=1)
y_hat.name = 'y_hat'

# COST
cost = softmax.categorical_cross_entropy(y, linear3, extra_ndim=1).mean()
cost.name = 'cost'

model = Model(cost)
# Load model parameters from a file
with open('reg1_best.tar') as model_file:
    model_params = model.get_parameter_dict().keys()
    param_vals = {k:v for k,v in load_parameters(model_file).iteritems() if k in model_params}
    model.set_parameter_values(param_vals)

#take activations of last element
activations = [h1[-1].flatten(), h2[-1].flatten()]
initial_states = [grnn1.parameters[-1], grnn2.parameters[-1]]
states_as_params = [tensor.vector(dtype=initial.dtype) for initial in initial_states]

#Get prob. distribution of the last element in the last seq of the batch
fprop = theano.function([x] + states_as_params, activations + [y_hat[-1, -1, :]], givens=zip(initial_states, states_as_params))

In [20]:
print stochastic_sampling(500, ix_to_char, u'bienaventurado ', temperature = 1.0)

Starting sampling
bienaventurado en la cama dalel
no vuelvas pa meterle la bocina no la miro es agatillo
me gustan más dando a sacarme
no inverturas conmigo no es el movimiento
en mi maquina reproyeda
y la posición contigo me muero
si en mi
dale candela

nicky jam
y tu quiere tomar a poco
acaricias yo te voy a dar lo que le gusto no pa la calle
y cadera la camisa la que me pare la estación
tranquilo y te necesite de música.

pídelo que no hay como la velo
tu cuerpo yo sigo choco
vólverán un día
cae mami, asi es cosa (oh!)
ella 
None


In [25]:
!grep -i "si tu quieres mami" reg1.txt 


recuerda si tu quieres mami
recuerda si tu quieres mami
recuerda si tu quieres mami
